In [ ]:
import os
import csv
import time
import random
import requests
from pathlib import Path
from textwrap import dedent

# base ship data URL
ship_url = 'https://maritime.ihs.com/Ships/Details/Index/'
# ship URLs for additional details
# ship_additional_data_urls = {
#     "data": "url",
# }

# session data - headers
session_headers = {
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
    "sec-ch-ua": "\" Not A;Brand\";v=\"99\", \"Chromium\";v=\"100\", \"Google Chrome\";v=\"100\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"macOS\"",
    "Upgrade-Insecure-Request": "1",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36" \
                  "(KHTML, like Gecko) Chrome/100.0.4896.60 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8," \
              "application/signed-exchange;v=b3;q=0.9",
    "host": "maritime.ihs.com",
    "Cookie": ".AspNet.ApplicationCookie=1NggUpzRSPm-L0alWgLuec9AhZOIvnseH18Rw8gbjE-qBTsWqCCHcniyMT9r5FdSUalmhJLHBQD8EjjNTJoLZR6OcbAbVyek16WQA7S4o9F2jXp7KQihe49X6t-h-3yNn9ByuUTnGouBmUnnhj08W-t_9tbJHu79CkSpvSQWD3S0TPRsenSZtq9zqLY8K5fKZ_388LDn6sawrKmNYXm0Zs4iQwENngAm92cVsRgFLA0bl9lgUP5Pqa_63UtV1MDLfRoYcyD8TrEmlCO3v__JY9bDxD9uaKmybrs7qpSkDlt1XMCiKin3Ju0qsciAiEy-luw48mthg9YXBklyXIB4a-uwJaF9016tZxix60QTfJk-2Cva82gpbRGAVBpRzlS9vGDgpVOexjXEKOhJDs9bdEYTyMj6GnrKwxo8_RmKHY6IDUL91O4erzn7Wh5I6EGjbHHmyWE8df7OaBq6Vu9Qv6gi8bdu8EtzW6YVdXKA1HK9ZtbIvurDPM8WdnIsd79b5WkOCaAubNbRX5Zug6KzTiXlzradIYzgCvBoSQmVfFFxy3CD; ADRUM_BT1=R:0|i:483940|e:23; ADRUM_BTa=R:0|g:8e85e0f3-8070-40a1-bb68-c09b6118c8b3|n:ihsco_86f4fc53-86e8-4feb-bc7a-7921d526821f; ASP.NET_SessionId=v51wd5xrpmrkdye0amopahko; BIGipServer~ProdWeb~pool-maritime.ihs.com-80=rd1100o00000000000000000000ffff0aa8006bo80; LastBuilderVisited=JPN138Miho Zosensho K.K.+RUS018Vyborgskiy Sudostroitelnyy Zavod%2c Oao; LastShipVisited=1009340A+8009129DF 19+9237371WEC VERMEER+9237369MOVEON+9334600ABAI+7920259ABDUL B; SameSite=None; ShipCommercialHistoryExpanded=ShipCommercialHistoryExpanded; ckDefault=page=shipsearch&records=20; ckShip=db_name001=VESSELNAMEBROWSE&en_name001=Name of Ship&db_name002=DATEOFBUILDBROWSE&en_name002=Built&db_name003=DWT&en_name003=Deadweight&db_name004=FLAG&en_name004=Flag&db_name005=STATUSBROWSE&en_name005=Status&db_name006=OWNER&en_name006=Registered Owner&db_name007=NBPriceUSDEquivalent&en_name007=Newbuilding Price&db_name008=EngineBuilderLargest&en_name008=Engine Builder&db_name009=EngineMakeLargest&en_name009=Engine Design&db_name010=SHIPBUILDER&en_name010=Shipbuilder&db_name011=BUILDERCODE&en_name011=Shipbuilder Code&db_name012=YEAROFBUILDBROWSE&en_name012=Year; ckShipDiv=hidehidehidehidehideshowhidehide; list=; rememberMeLogin=C85751F4D8BAC8E64E7C1278E492E3F616BC06CAD6FE828D8DD1CED1D48892E81320AE0853EE3F6B5BF47B6B82CBA646A7CD8EA72472A24EDEFA6A78F14AABFA34F9C3001C95D0C472E89C263A6B2D1138EA70FF73A2EA721BAE40B17B83539F14889F7FB612664A05B31FCD; shipsearch=",
}

# session data - cookies
session_cookies = {
    # these two cookies are the most important ones for the scraping
    ".AspNet.ApplicationCookie": "tDpBsG1oYSJIc_3Vb2geMLwS1EdEi8CceoZsq8p3i27RVfqQFiGEIGdwA3P3e9BduhX90QmJLe40P6rBGrm8EpRF6qtq6URvSoMMrDFMp15mz6WkwNPqE2TKqm87ENzQ_rGgwnptOcAm03TcokUgajbG1KGGnyuZyiBwVRGlHRvkiWpH8N-Qq4SI8cmdwk6nobUx6oW80cFHL1A1duolWZxr9d9E-fNnQWO-LwzL3Oz4F8Sxn7uVujUUT91ELWvvWGtWLoz2i7By2jThKlznyl57hzo4spj-f9RirHprTmHoj5RetwBrEfYRpLw2Ukd8FQDG4Z6DdikeMNW-v_ZSAcHPc1YWaT7_lFTblwiUtL3n6AGhh2ygWNZOz_ZYDIxHFEJuUupOhol--GiVzVRLbK0mqptdSm2nmk9Rj-fcUn7Jgr0SOIvgkJCESu4U0U9AjPlJlmn3L79sRe5XT2V6ScA0f_8LjnMJalrl8HiQY4JIaCACvK0bJ_wFzrNlkOOeZRwcfi64TampIsqk6SC-n7Z9jq4sOEQcg8nO_V8q8qiFv_S0",    
    "ASP.NET_SessionId": "vgw3qjc0jnenb2fzbeu0wzrc",
    
    "BIGipServer~ProdWeb~pool-maritime.ihs.com-80": "rd1100o00000000000000000000ffff0aa8006bo80",
    "LastBuilderVisited": "JPN138Miho Zosensho K.K.+RUS018Vyborgskiy Sudostroitelnyy Zavod%2c Oao",
    "LastShipVisited": "1009340A+8009129DF 19+9237371WEC VERMEER+9237369MOVEON+9334600ABAI+7920259ABDUL B",
    "ShipCommercialHistoryExpanded": "ShipCommercialHistoryExpanded",
    "ckDefault": "page=shipsearch&records=20",
    "ckShip": "db_name001=VESSELNAMEBROWSE&en_name001=Name of Ship&db_name002=DATEOFBUILDBROWSE&en_name002=Built&" \
              "db_name003=DWT&en_name003=Deadweight&db_name004=FLAG&en_name004=Flag&db_name005=STATUSBROWSE&" \
              "en_name005=Status&db_name006=OWNER&en_name006=Registered Owner&db_name007=NBPriceUSDEquivalent&" \
              "en_name007=Newbuilding Price&db_name008=EngineBuilderLargest&en_name008=Engine Builder&" \
              "db_name009=EngineMakeLargest&en_name009=Engine Design&db_name010=SHIPBUILDER&" \
              "en_name010=Shipbuilder&db_name011=BUILDERCODE&en_name011=Shipbuilder Code&" \
              "db_name012=YEAROFBUILDBROWSE&en_name012=Year",
    "ckShipDiv": "hidehidehidehidehideshowhidehide", 
    "list": "",
    "rememberMeLogin": "C85751F4D8BAC8E64E7C1278E492E3F616BC06CAD6FE828D8DD1CED1D48892E81320AE0853EE3F6B5BF" \
                       "47B6B82CBA646A7CD8EA72472A24EDEFA6A78F14AABFA34F9C3001C95D0C472E89C263A6B2D1138EA70FF73" \
                       "A2EA721BAE40B17B83539F14889F7FB612664A05B31FCD",
    "shipsearch": "",
    "shiphead": "1",
}


#print(session_cookies)
#print(session_headers)

# setup sessions
session = requests.Session()
session.headers.update(session_headers)
#session.cookies.update(session_cookies)

#session.max_redirects = 100
#print(session.cookies, "\n")

# request the site
#response = session.get(ship_url, allow_redirects=True)
#print(response.status_code)
#print(response.text)


# # raw data store (for all the data)
# raw_data = os.getcwd() + "/seaweb"
# print(f"raw data store = {raw_data}")

with open('EquasisToIACS_20220401_731.csv') as csv_file:  # read CSV from equasis with IMO numbers
    csv_reader = csv.reader(csv_file, delimiter=';')

    line_count = 0
    for row in csv_reader:  # iterate over all IMO numbers
        print(f"Total processed: {line_count}")  # just a debug
        
        if line_count > 100000:  # just a stopper
            break
            
        if line_count == 0:  # skip the header row
            print(f'Column names are {", ".join(row)}')
            line_count += 1
            with open('processed.csv', mode='w') as processed_file:  # rewrite existing file
                processed_writer = csv.writer(processed_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                processed_writer.writerow([f"{row[0]}", "Processed"])
        
        else:  # data rows processing
            print(f'\nProcessing: IMO = {row[0]}, SHIP NAME = {row[1]}')
            line_count += 1
                
            ship_dir = os.getcwd() + "/seaweb/" + row[0]  # directory to store the current ship
            print(f"\tship dir: {ship_dir}")
            
            with open('processed.csv', mode='a') as processed_file:  # other mode='w'
                processed_writer = csv.writer(processed_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                processed_writer.writerow([f"{row[0]}", "+"])
                
            if Path(ship_dir).exists() and Path(ship_dir).is_dir(): # skip already processed ships
                print(f"\tShip IMO: {row[0]} already processed. Skipped.")
                continue
            
            # artificial delay for every 50 run
            if line_count % 50 == 0:
                delay_sec = random.randint(2, 5)  # delay 1 to 3 sec (both inclusive)
                print(f"\tDelay {delay_sec} seconds.")
                time.sleep(delay_sec)
        
            # real ship processing
            response = session.get(ship_url + row[0], allow_redirects=True)  # request the site...
            if response.status_code != 200:  # check that 200 is returned
                print(f"Error code: {response.status_code} returned! Stopping!")
                break
            
            os.makedirs(ship_dir, exist_ok=True)  # if all is OK - create dir for the ship data
            with open(Path(ship_dir + '/ship_main.html'),'w') as f:  # write received content to file
                f.write(response.text)
                print(f"\tWritten file: {ship_dir + '/ship_main.html'}")
            

    print(f'Processed {line_count} lines.')

